<a href="https://colab.research.google.com/github/sathish1313/Brain-Tumor/blob/main/Brain_Tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

import matplotlib.pyplot as plt

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
data_dir = '/content/drive/MyDrive/DATASET/Brain_Tumor_Detection'

In [ ]:
label_mapping = {'no': 0, 'yes': 1}

In [ ]:
import os
images = []
labels = []
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        label = label_mapping.get(folder)
        if label is not None:
            for file_name in os.listdir(folder_path):
                if file_name.endswith(".jpg"):
                    file_path = os.path.join(folder_path, file_name)
                    image = load_img(file_path, color_mode="grayscale", target_size=(64, 64))
                    image_array = img_to_array(image)
                    images.append(image_array)
                    labels.append(label)

In [ ]:

images = np.array(images)
labels = np.array(labels)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
num_timesteps = X_train.shape[1]
height = X_train.shape[2]
width = X_train.shape[3]
input_dim = 1

In [ ]:
X_train = X_train.reshape(-1, num_timesteps, height * width)
X_test = X_test.reshape(-1, num_timesteps, height * width)

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(num_timesteps, height * width)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.7147 - loss: 0.5589 - val_accuracy: 0.7050 - val_loss: 0.5667
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.7347 - loss: 0.5310 - val_accuracy: 0.7100 - val_loss: 0.5437
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.7497 - loss: 0.5040 - val_accuracy: 0.7250 - val_loss: 0.5217
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.7530 - loss: 0.4872 - val_accuracy: 0.7417 - val_loss: 0.4687
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.7662 - loss: 0.4719 - val_accuracy: 0.7133 - val_loss: 0.5828
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.7680 - loss: 0.4954 - val_accuracy: 0.7783 - val_loss: 0.4506
Epoch 7/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.7957 - loss: 0.4363 - val_accuracy: 0.7600 - val_loss: 0.4506
Epoch 8/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.8182 - loss: 0.4103 - val_accuracy: 0.7933 - v

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9806 - loss: 0.0623
Test Loss: 0.09175185114145279
Test Accuracy: 0.9800000190734863


In [ ]:
def predict_single_image(image_path):
    image = load_img(image_path, color_mode="grayscale", target_size=(64, 64))
    image_array = img_to_array(image) / 255.0
    image_array = image_array.reshape(1, num_timesteps, height * width)
    prediction = model.predict(image_array)
    return prediction[0][0] > 0.5

In [ ]:
test_directory_path = '/content/drive/MyDrive/DATASET/Brain_Tumor_Detection/pred'

def analyze_images_in_directory(directory_path):
    image_files = [f for f in os.listdir(directory_path) if f.endswith(".jpg")]
    if not image_files:
        print("No images found in the directory.")
    for file_name in image_files:
        file_path = os.path.join(directory_path, file_name)
        try:
            prediction = predict_single_image(file_path)
            print(f"Image: {file_name}, Tumor present: {'Yes' if prediction else 'No'}")
        except Exception as e:
            print(f"Error processing image {file_name}: {e}")
            analyze_images_in_directory(test_directory_path)

In [ ]:
analyze_images_in_directory(test_directory_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
Image: pred18.jpg, Tumor present: No
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Image: pred2.jpg, Tumor present: Yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Image: pred24.jpg, Tumor present: No
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Image: pred22.jpg, Tumor present: Yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Image: pred13.jpg, Tumor present: Yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Image: pred20.jpg, Tumor present: No
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Image: pred16.jpg, Tumor present: Yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Image: pred17.jpg, Tumor present: No
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Image: pred15.jpg, Tumor present: No
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Image: pred1.jpg, Tumor present: No
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Image: pred0.jpg, Tumor present: No
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Image: pred23.jpg, Tumor present: Yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Image: pred25.jpg, Tumor present: No
1/1 ━━━━━━━━━━━━━━━━━━